# Imports

In [1]:
# We will use to this to evaluate the ROUGE score of the model
!pip install evaluate &> /dev/null
!pip install rouge-score &> /dev/null

The syntax of the command is incorrect.
The syntax of the command is incorrect.


In [ ]:
import math
import re
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler
from matplotlib import pyplot as plt

# import evaluate
# rouge = evaluate.load('rouge')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# Dataset Processing

In [ ]:
# Download the dataset (English to German)
!wget https://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip

with open("deu.txt") as f:
  sentences = f.readlines()

print(f"Dataset size: {len(sentences)}")
sentences[:5]

--2023-03-31 13:12:07--  https://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9780074 (9.3M) [application/zip]
Saving to: ‘deu-eng.zip.1’

deu-eng.zip.1       100%[===================>]   9.33M  4.72MB/s    in 2.0s    

2023-03-31 13:12:10 (4.72 MB/s) - ‘deu-eng.zip.1’ saved [9780074/9780074]

Archive:  deu-eng.zip
replace deu.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: deu.txt                 
  inflating: _about.txt              
Dataset size: 260434


['Go.\tGeh.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)\n',
 'Hi.\tHallo!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)\n',
 'Hi.\tGrüß Gott!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)\n',
 'Run!\tLauf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)\n',
 'Run.\tLauf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #941078 (Fingerhut)\n']

In [ ]:
###############
# TODO: Replace this with our data
###############

NUM_INSTANCES = 10000 # Dataset size limit
MAX_SENT_LEN = 12 # Maximum input sequence length
MAX_OUTPUT_LEN = 15 # Maximum output sequence length

# Used as English and German datasets
eng_sentences, deu_sentences = [], []

# Vocabluary for input encoder (English) and output decoder (German) 
eng_words, deu_words = set(), set()

for i in range(NUM_INSTANCES):
  
  # Pick a random sample
  rand_idx = np.random.randint(len(sentences))

  # Tokenize the data (this is very bad and we probably want to use an actual tokenizer like spacy)
  eng_sent, deu_sent = ["<sos>"], ["<sos>"]
  eng_sent += re.findall(r"\w+", sentences[rand_idx].split("\t")[0]) 
  deu_sent += re.findall(r"\w+", sentences[rand_idx].split("\t")[1])

  # change to lowercase
  eng_sent = [x.lower() for x in eng_sent]
  deu_sent = [x.lower() for x in deu_sent]

  eng_sent.append("<eos>")
  deu_sent.append("<eos>")

  if len(eng_sent) >= MAX_SENT_LEN:
    eng_sent = eng_sent[:MAX_SENT_LEN]
  else:
    for _ in range(MAX_SENT_LEN - len(eng_sent)):
      eng_sent.append("<pad>")

  if len(deu_sent) >= MAX_OUTPUT_LEN:
    deu_sent = deu_sent[:MAX_OUTPUT_LEN]
  else:
    for _ in range(MAX_OUTPUT_LEN - len(deu_sent)):
      deu_sent.append("<pad>")

  # Add parsed sentences
  eng_sentences.append(eng_sent)
  deu_sentences.append(deu_sent)

  # Update vocabluary
  eng_words.update(eng_sent)
  deu_words.update(deu_sent)

# Convert vocabluaries into indexable-lists
eng_words, deu_words = list(eng_words), list(deu_words)

# Convert tokens to indicies 
for i in range(len(eng_sentences)):
  eng_sentences[i] = [eng_words.index(x) for x in eng_sentences[i]]
  deu_sentences[i] = [deu_words.index(x) for x in deu_sentences[i]]
  if i % 5000 == 0:
    print(f"{i:05}/{len(eng_sentences)} done")

idx = 10
print(eng_sentences[idx])
print([eng_words[x] for x in eng_sentences[idx]])
print(deu_sentences[idx])
print([deu_words[x] for x in deu_sentences[idx]])

00000/50000
05000/50000
10000/50000
15000/50000
20000/50000
25000/50000
30000/50000
35000/50000
40000/50000
45000/50000

[5984, 5711, 111, 7654, 4377, 2755, 2755, 2755, 2755, 2755, 2755, 2755]
['<sos>', 'tom', 'avoided', 'mary', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
[10697, 10197, 14692, 7549, 10888, 7771, 4769, 4769, 4769, 4769, 4769, 4769, 4769, 4769, 4769]
['<sos>', 'tom', 'wich', 'maria', 'aus', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self):
    self.source = np.array(eng_sentences, dtype = int)
    self.target = np.array(deu_sentences, dtype = int)
    
  def __getitem__(self, idx):
    # get item by index
    return self.source[idx], self.target[idx]
  
  def __len__(self):
    # returns length of data
    return len(self.source)

dataset = Dataset()

# We we sample from this dataset by sampling from the indices
indices = list(range(len(dataset)))
test_idx = np.random.choice(indices, size = int(len(dataset) * 0.1), replace = False)
train_idx = list(set(indices) - set(test_idx))
val_idx = np.random.choice(train_idx, size = int(len(dataset) * 0.1), replace = False)
train_idx = list(set(train_idx) - set(val_idx))
train_sampler, val_sampler, test_sampler = SubsetRandomSampler(train_idx), SubsetRandomSampler(val_idx), SubsetRandomSampler(test_idx)

train_loader = torch.utils.data.DataLoader(dataset, batch_size = 128, sampler = train_sampler)
val_loader = torch.utils.data.DataLoader(dataset, batch_size = 128, sampler = val_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size = 128, sampler = test_sampler)

In [ ]:
print(f"Train dataset size: {len(train_idx)}")
print(f"Val dataset size: {len(val_idx)}")
print(f"Test dataset size: {len(test_idx)}")
print(f"English vocab size: {len(eng_words)}")
print(f"German vocab size: {len(deu_words)}")
x, y = next(iter(train_loader)) # Get a batch
x, y

Train dataset size: 40000
Val dataset size: 5000
Test dataset size: 5000
English vocab size: 9116
German vocab size: 16301


(tensor([[5984, 5711, 2576,  ..., 4377, 2755, 2755],
         [5984, 5711, 4044,  ..., 4377, 2755, 2755],
         [5984, 5711, 8592,  ..., 2755, 2755, 2755],
         ...,
         [5984, 3027, 3969,  ..., 2755, 2755, 2755],
         [5984, 5711,  999,  ..., 2755, 2755, 2755],
         [5984, 4119, 6357,  ..., 2755, 2755, 2755]]),
 tensor([[10697, 10197, 11858,  ...,  4769,  4769,  4769],
         [10697, 10197,  9397,  ...,  4769,  4769,  4769],
         [10697, 10197,  5784,  ...,  4769,  4769,  4769],
         ...,
         [10697, 11598, 10604,  ...,  4769,  4769,  4769],
         [10697,  2488,   461,  ...,  4769,  4769,  4769],
         [10697, 14881, 12236,  ...,  4769,  4769,  4769]]))

# Transformer Model

In [ ]:
class PositionalEncoding(nn.Module):
    """
    Classic Attention-is-all-you-need positional encoding.
    Source: PyTorch docs: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


def generate_square_subsequent_mask(size):
    """
    Generates an upper-triangular matrix of -inf, with zeros on diag.
    Source: PyTorch docs: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """
    return torch.triu(torch.ones(size, size) * float('-inf'), diagonal=1)


class Transformer(nn.Module):
    """
    Transformer model with both encoder and decoder (seq2seq).
    Prediction-time inference is done greedily (argmax of logits).
    """

    def __init__(self, enc_vocab_size, dec_vocab_size, nhead, num_layers, hidden_dim, embedding_dim, max_output_length):
        super().__init__()

        self.dim = embedding_dim
        self.max_output_length = max_output_length

        # Encoder setup
        self.enc_embedding = nn.Embedding(enc_vocab_size, embedding_dim)
        self.pos_encoder = PositionalEncoding(d_model=self.dim)
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(d_model=self.dim, nhead=nhead, dim_feedforward=hidden_dim),
            num_layers=num_layers
        )

        # Decoder setup
        self.dec_embedding = nn.Embedding(dec_vocab_size, embedding_dim)
        self.y_mask = generate_square_subsequent_mask(self.max_output_length)
        self.transformer_decoder = nn.TransformerDecoder(
            decoder_layer=nn.TransformerDecoderLayer(d_model=self.dim, nhead=nhead, dim_feedforward=hidden_dim),
            num_layers=num_layers
        )
        self.fc = nn.Linear(self.dim, dec_vocab_size)

        # Init embedding and fc weights
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.1
        self.enc_embedding.weight.data.uniform_(-initrange, initrange)
        self.dec_embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        self.fc.weight.data.uniform_(-initrange, initrange)
      
    def forward(self, x, y) :
        """
        Compute forward pass through transformer.
        """
        encoded_x = self.encode(x)
        output = self.decode(y, encoded_x)
        return output.permute(1, 2, 0)

    def encode(self, x):
        """
        Passes x through the transformer encoder.
        """
        x = x.permute(1, 0)
        x = self.enc_embedding(x) * math.sqrt(self.dim)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        return x

    def decode(self, y, memory):
        """
        Given memory (encoder output), computes decoder output.
        """
        y = y.permute(1, 0)
        y = self.dec_embedding(y) * math.sqrt(self.dim)
        y = self.pos_encoder(y)
        y_mask = self.y_mask[:y.shape[0], :y.shape[0]].type_as(memory) # slice the mask
        output = self.transformer_decoder(y, memory, y_mask)
        output = self.fc(output)
        return output

    def predict(self, x):
        """
        Method to use at inference/test time. Predict y from x one token at a time. This method is greedy
        decoding (argmax of logits).
        """
        memory = self.encode(x)
        output_tokens = (torch.ones((x.shape[0], self.max_output_length))).type_as(x).long()
        output_tokens[:, 0] = deu_words.index("<sos>") # Initialize with SOS index
        for o in range(1, self.max_output_length):
            y = output_tokens[:, :o] 
            output = self.decode(y, memory)
            output = torch.argmax(output, dim=-1)
            output_tokens[:, o] = output[-1:]  # Get the last output token
        return output_tokens

# Model Hyperparameter Settings
ENG_VOCAB_SIZE = len(eng_words) # English vocab size - needed for encoder embedding
DEU_VOCAB_SIZE = len(deu_words) # German vocab size - needed for decoder embedding
EMBEDDING_DIM = 512 # Dimension for embedding for both encoder and decoder embeddings
HIDDEN_DIM = 1024 # Number of units in feedforward network model for both encoder and decoder layers
NUM_HEADS = 8 # Number of multihead attention models
NUM_LAYERS = 4 # Number of sub-encoder/sub-decoder layers in encoder/decoder
DROPOUT = 0.3 # Dropout for positional encoding

# Create model
model = Transformer(ENG_VOCAB_SIZE, DEU_VOCAB_SIZE, NUM_HEADS, NUM_LAYERS, HIDDEN_DIM, EMBEDDING_DIM, MAX_OUTPUT_LEN)
model.to(DEVICE)
pass # pass to avoid outputting model summary

# Training

In [ ]:
criterion = nn.CrossEntropyLoss()
NUM_EPOCHS = 100
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

train_loss_trace = []
val_loss_trace = []

for epoch in range(1, NUM_EPOCHS+1):
  train_loss = 0
  train_total = 0
  for i, (x, y) in enumerate(train_loader):
    x, y  = x.to(DEVICE), y.to(DEVICE)
    logits = model(x, y[:, :-1])
    loss = criterion(logits, y[:, 1:])
    train_loss += loss.item()
    train_total += x.shape[0]

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5, norm_type=2.0) # Gradient clipping
    optimizer.step()

  train_loss_trace.append(train_loss / train_total)

  val_loss = 0
  val_total = 0
  for j, (val_x, val_y) in enumerate(val_loader):
    val_x, val_y  = val_x.to(DEVICE), val_y.to(DEVICE)
    val_loss += criterion(model(val_x, val_y[:, :-1]), val_y[:, 1:]).item()
    val_total += val_x.shape[0]
  val_loss_trace.append(val_loss / val_total)

  if epoch % 5 == 0:
    print(f"Epoch {epoch:03} | Train Loss: {train_loss:02.5f} | Val Loss: {val_loss:02.5f}")

  torch.save(model.state_dict(), f"./model-e{epoch}.pth")

# loss curve
plt.plot(range(1, NUM_EPOCHS+1), train_loss_trace, 'r-')
plt.plot(range(1, NUM_EPOCHS+1), val_loss_trace, 'orange')
plt.xlabel('Epoch')
plt.ylabel('Avg. Loss')
plt.show()

Epoch 000 | Train Loss: 6.98611 | Val Loss: 1.00325
Epoch 005 | Train Loss: 2.44687 | Val Loss: 0.56119
Epoch 010 | Train Loss: 1.19124 | Val Loss: 0.53306
Epoch 015 | Train Loss: 0.57891 | Val Loss: 0.51598
Epoch 020 | Train Loss: 0.30850 | Val Loss: 0.45541
Epoch 025 | Train Loss: 0.19848 | Val Loss: 0.51817
Epoch 030 | Train Loss: 0.14967 | Val Loss: 0.46885
Epoch 035 | Train Loss: 0.12326 | Val Loss: 0.49360
Epoch 040 | Train Loss: 0.10959 | Val Loss: 0.59720
Epoch 045 | Train Loss: 0.09647 | Val Loss: 0.61238
Epoch 050 | Train Loss: 0.08921 | Val Loss: 0.63954
Epoch 055 | Train Loss: 0.08100 | Val Loss: 0.63095
Epoch 060 | Train Loss: 0.07854 | Val Loss: 0.57894


# Testing (ROUGE-L score)

Computer the average ROUGE-L score (higher is better). Typically scores of 50 is considered good.

In [ ]:
def get_sentence(tokens, vocab):
  """
  Given token indicies and vocab, convert tokens back to words in vocab.
  """
  return [vocab[x] for x in tokens]

# def get_avg_rouge_score(model, test_loader):
#   """
#   Compute the average ROUGE-L metric given the test dataset.
#   In general, ROUGE-L compares how "similar" the prediction is compared to the expected output
#   """
#   model.eval()
#   avg_rouge_score = 0
#   for i, (x, y) in enumerate(test_loader):

#     x = x.to(DEVICE)
#     y = y.to(DEVICE)

#     logits = model(x, y[:, :-1])
#     pred = model.predict(x)

#     test_sentences = []
#     pred_sentences = []

#     for j in range(y.shape[0]):
#       test_sentences.append([" ".join(get_sentence(y[j, :], deu_words))])
#       pred_sentences.append(" ".join(get_sentence(pred[j, :], deu_words)))

#     results = rouge.compute(predictions=pred_sentences, references=test_sentences)
#     avg_rouge_score += results['rougeL'] 
  
#   return avg_rouge_score

# print(f"Test dataset Average ROUGE-L score: {get_avg_rouge_score(model, test_loader)}")

Test dataset Average ROUGE-L score: 17.288985546095212


# Inference

In [ ]:
src, tgt = next(iter(test_loader))
src = src.to(DEVICE)
tgt = tgt.to(DEVICE)
print(f"Input: {' '.join(get_sentence(src[0], eng_words))}")
print(f"Expected Output: {' '.join(get_sentence(tgt[0], deu_words))}")

model.eval()
outputs = model.predict(src[0,].unsqueeze(0))
print(f"Actual Output: {' '.join(get_sentence(outputs.squeeze(0), deu_words))}")

Input: <sos> tom concealed the fact that he had been in prison <eos>
Expected Output: <sos> tom verschwieg die tatsache dass er im gefängnis war <eos> <pad> <pad> <pad> <pad>
Actual Output: <sos> tom mondes heute abend die ganze woche in der schule <eos> <pad> <pad> <pad>
